## Mini Exploration - Calculating Complex Power

Lets calculate complex power at multiple points in time using the sunshine data. You can read more about the sunshine dataset on the blog [here](https://blog.ni4ai.org/post/2020-03-30-sunshine-data/). You can find a diagram with approximate sensor locations [here](https://blog.ni4ai.org/post/2021-05-29-disaggregation/). [This blog post](https://blog.ni4ai.org/post/2021-06-19-power_factor/) introduces what *complex power* is, and of course you can find out more on wikipedia. Another relevant topic is the meaning of phasors which is introduced [here](https://blog.ni4ai.org/post/2020-07-30-what-is-the-angle/). 

The basic calculation is as follows: $S = VI^*$
 [in this equation all three quantities are complex numbers ie phasors]. What we store in the `btrdb` platform is the magnitude and angle of a phasor, which you can also rewrite as the real and imaginary components, by Euler’s formula.
 

Functions/objects used here:
```
btrdb.connect()
numpy.unwrap()
btrdb.streams.StreamSet
btrdb.streams.StreamSet.earliest()
btrdb.streams.StreamSet.latest()
btrdb.streams.StreamSet.values()
btrdb.point.RawPoint
```

# Imports

In [ ]:
import pandas as pd
import btrdb
import numpy as np
import re
from btrdb.utils.timez import *
from tabulate import tabulate
from pprint import pprint
from typing import List, Tuple
from matplotlib import pyplot as plt

# Connect To Server
To get started we'll connect to the server and define a helper method from a previous notebook to describe our `btrdb.Stream`s.

In [ ]:
connection = btrdb.connect()
pprint(connection.info())

In [ ]:
# add a helper method to describe streams from streamset tutorial notebook 3
def describe_streams(streams: btrdb.stream.StreamSet):
    table = [["Collection", "Name", "Units", "Version", "Earliest", "Latest"]]
    for stream in streams:
        tags = stream.tags()
        table.append(
            [
                stream.collection,
                stream.name,
                tags["unit"],
                stream.version(),
                stream.earliest()[0].time,
                stream.latest()[0].time,
            ]
        )
    return tabulate(table, headers="firstrow")

Lets take a look at the [sunshine data set again](https://blog.ni4ai.org/post/2020-03-30-sunshine-data/).

In [ ]:
sunshine_collection = connection.list_collections("sunshine")
pprint(sunshine_collection)

For the complex power calculations, let first look at a single collection in this list. In this case, `PMU1`.

In [ ]:
pmu1 = connection.streams_in_collection(sunshine_collection[0])
print(describe_streams(pmu1))

`StreamSet`s provide us with a nice way to operate on many `Stream`s at once. Lets convert `pmu1` to a `StreamSet`

In [ ]:
pmu1_streamset = btrdb.stream.StreamSet(pmu1)
pprint(pmu1_streamset)

StreamSets are iterable objects, so our `describe_streams` method we defined above still works!

In [ ]:
print(describe_streams(pmu1_streamset))

Lets take a look at when the last data point was added to each `Stream`

In [ ]:
# when was the latest datapoint added?
btrdb.utils.timez.ns_to_datetime(pmu1_streamset[0].latest()[0][0])

# Viewing Data

Like the `Stream` object, the `StreamSet` has a `values` method which will return a list of lists.  Each internal list contains the `RawPoint` instances for a given stream.

---

Lets add a helper method to take a `RawPoint` and return the time value from it in nanoseconds

In [ ]:
def ns_from_rawpoint(point: btrdb.point.RawPoint) -> int:
    """Automatically take a rawpoint and return the time value (in ns)
    when the data was added."""

    return point[0][0]

One more helper method to parse `RawPoint`s and convert them into numpy arrays, since we will be using these values a lot.

In [ ]:
def values_to_arr(values: Tuple[btrdb.stream.RawPoint]) -> np.array:
    "Convert a tuple of RawPoints into a numpy array of time, value pairs."
    return np.asarray([[time, val] for time, val in values])

To make things a bit simpler, we know that PMU devices excel in time-synchronized data, so we will take a few liberties in data valdiation and assume that the `RawPoint`s we return from our streamset will be equally sized for each stream.

When was the latest data point added to each stream in sunshine/PMU1 collection?

In [ ]:
last_time_updated = []
for stream in pmu1_streamset:
    tmp_point = stream.latest()
    last_time_updated.append(
        [stream.tags().get("name", "N/A"), ns_from_rawpoint(tmp_point)]
    )
for val in last_time_updated:
    print(f"Stream: {val[0]}\t Last Updated: {ns_to_datetime(val[1])}")

Here is an equivalent way to do the same as above, but using a `StreamSet` method.

In [ ]:
for stream, time in zip(pmu1_streamset, pmu1_streamset.latest()):
    print(
        f"Stream: {stream.tags().get('name')}\t Last Updated: {ns_to_datetime(time.time)}"
    )

Lets find the maximum starting time (in ns) and calculate the end time for our power calculation. Lets return all values from our start point to 100 seconds after our start point.

In [ ]:
start_time_arr = values_to_arr(pmu1_streamset.earliest())

start = np.max(start_time_arr[:, 0])
end = start + ns_delta(seconds=100)
print(start)
print(end)

print(f"our bounding start time will be: {btrdb.utils.timez.ns_to_datetime(start)}")
print(f"our end time will be: {btrdb.utils.timez.ns_to_datetime(end)}")

# Complex Power

These PMU datasets are exactly what we need to calculate the instantaneous power over some duration of time.

To calculate this, lets look again at the equation for instantaneous power:
The basic calculation is as follows: $S = VI^*$

Our voltage $V$ is a Phasor with a magnitude and an imaginary portion. This is represented by the magnitude and angle of these measurements and the individual streams named:
`L1MAG` to `L3MAG` and `L1ANG` to `L3ANG` respectively. The same can be said for the current $I$ as well, with the streams named: `C1MAG` to `C3MAG` and `C1ANG` to `C3ANG` respectively.


# Filtering StreamSets

Since we know that the voltage and current streams in our stream set each have their corresponding angles and magnitudes, lets group these into more logical `StreamSet`s by filtering these such that all of our current and voltage related streams are grouped together. We can use regex matching to do some advanced filtering.

In [ ]:
pmu1_c = pmu1_streamset.filter(name=re.compile("C[\d]"))
pmu1_l = pmu1_streamset.filter(name=re.compile("L[\d]"))

print(describe_streams(pmu1_c))
print()
print(describe_streams(pmu1_l))

Lets take these `StreamSet`s and and convert the range of values we want to `pandas.DataFrame`s

In [ ]:
pmu1_voltage_df = pmu1_l.filter(start=start, end=end).to_dataframe()
pmu1_current_df = pmu1_c.filter(start=start, end=end).to_dataframe()

display(pmu1_voltage_df)
display(pmu1_current_df)

Lets convert our magnitude and angles to their complex forms for each combination of streams.

**Remember** that we also need to unwrap the angles and convert them to radians.

In [ ]:
prefix = "sunshine/PMU1/"
mag = "MAG"
ang = "ANG"
for i in range(1, int(len(pmu1_c) / 2) + 1):
    # convert to radians and unwrap the angles
    pmu1_current_df[f"{prefix}C{i}{ang}"] = np.unwrap(
        np.deg2rad(pmu1_current_df[f"{prefix}C{i}{ang}"][:])
    )
    pmu1_current_df[f"{prefix}{i}_complex_current"] = pmu1_current_df[
        f"{prefix}C{i}{mag}"
    ] * np.exp(1j * pmu1_current_df[f"{prefix}C{i}{ang}"])
display(pmu1_current_df)

In [ ]:
for i in range(1, int(len(pmu1_voltage_df.columns) / 2) + 1):
    # convert to radians and unwrap the angles
    pmu1_voltage_df[f"{prefix}L{i}{ang}"] = np.unwrap(
        np.deg2rad(pmu1_voltage_df[f"{prefix}L{i}{ang}"][:])
    )
    pmu1_voltage_df[f"{prefix}{i}_complex_voltage"] = pmu1_voltage_df[
        f"{prefix}L{i}{mag}"
    ] * np.exp(1j * pmu1_voltage_df[f"{prefix}L{i}{ang}"])
display(pmu1_voltage_df)

Lets calculate the complex power for each of the 3 sets of stream data we just created.

In [ ]:
col_list = [
    "sunshine/PMU1/1_complex_",
    "sunshine/PMU1/2_complex_",
    "sunshine/PMU1/3_complex_",
]
power_list = []
for col in col_list:
    power_list.append(
        pmu1_voltage_df[f"{col}voltage"][:]
        * np.conjugate(pmu1_current_df[f"{col}current"])
    )

Lets visualize these using `matplotlib.pyplot`

In [ ]:
plt.plot(power_list[0])
plt.xlabel("Time [ns]")
plt.ylabel("Power")

In [ ]:
plt.plot(power_list[1])
plt.xlabel("Time [ns]")
plt.ylabel("Power")

In [ ]:
plt.plot(power_list[2])
plt.xlabel("Time [ns]")
plt.ylabel("Power")